## K-Means Clustering Algorithm
The dataset is [Palmer Archipelago (Antarctica) penguin data](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data) which has 6 features and 1 label called species (Chinstrap, Adélie, or Gentoo)  
The dataset is saved in the "data/penguins_size.csv" file and preprocessed into x_train, x_test, y_train, y_test  
- Build a K-Means clustering algorithm to cluster the preprocessed data  
- Standardize the data and train the model with the training set
- Evaluate the model and print the confusion matrixes with both training and test sets


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# load the dataset
data = pd.read_csv('data/penguins_size.csv')
data.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [3]:
# data preprocessing
data = data.dropna()
data = data[data['sex'] != '.']

cleanup_nums = {"species": {"Adelie": 0, "Chinstrap": 1, "Gentoo": 2},
                "island": {"Biscoe": 0, "Dream": 1, "Torgersen": 2},
                "sex": {"MALE": 0.0, "FEMALE": 1.0}}
data = data.replace(cleanup_nums)

data.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,0.0
1,0,2,39.5,17.4,186.0,3800.0,1.0
2,0,2,40.3,18.0,195.0,3250.0,1.0
4,0,2,36.7,19.3,193.0,3450.0,1.0
5,0,2,39.3,20.6,190.0,3650.0,0.0


In [4]:

x = np.array(data.drop(['species'], axis=1).copy())
y = np.array(data['species'].copy()).astype(int)

In [5]:
#Data standardization x=(x-mean)/standard deviation.
x = (x-x.mean(axis = 0)) / (x.std(axis = 0))

In [6]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((266, 6), (67, 6), (266,), (67,))

In [7]:
# calculate the confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.cluster import pair_confusion_matrix

def evaluator(y_test, y_pred):    
    print()
    print("           CONFUSION MATRIX")
    print('actual      [            ]')
    print('              prediction ')
    print()
    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

    #print("                      EVALUATION METRICS")
    #print(classification_report(y_test, y_pred))



In [8]:
# setup a baseline model
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) # n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)


           CONFUSION MATRIX
actual      [            ]
              prediction 

Confusion matrix:
 [[  0  47  60]
 [  0  26  32]
 [101   0   0]]

           CONFUSION MATRIX
actual      [            ]
              prediction 

Confusion matrix:
 [[ 0 26 13]
 [ 0  8  2]
 [18  0  0]]


In [9]:
# build K-means model and only Numpy can be used
#assign each input to closest centroid and compute mean of each cluster 
#until error no longer decreases

class KMeans(object):
    def __init__(self):
        self.history = {}

    def initializekPrototypes(self, x, k):
        #randomly intialize k centroids
        samples, features = x.shape 
        w_j = np.empty((k, features))

        for i in range(k):
            random_idx = np.random.randint(0, samples - 1)
            w_j[i] = x[random_idx]

        return w_j


    def findClosestPrototype(self, xi, w_j, k):
        distancesToAllPrototypes = np.empty(k)
        for i in range(k):
            #calculate euclidian distance to help find closest centroid
            distancesToAllPrototypes[i] = np.sqrt(np.sum((xi - w_j[i])**2))

        #use argmin to return index with shortest distance
        return np.argmin(distancesToAllPrototypes, axis=0)
    
    
    def train(self, x, y, x_test, y_test, k):
        self.w_j = self.initializekPrototypes(x, k)
        m = x.shape[0]
        c_j = np.empty(m)
        previous_error = 1000000000
        iterations = 0

        while True:
          
           #assign each datapoint to a centroid 
            for i in range(m):
                c_j[i] = self.findClosestPrototype(x[i], self.w_j, k)
                
            #for each cluster - mean is new prototype
            #calculate error for each cluster 
            new_error = 0
            for i in range(k):
                cluster_idx =  np.where(c_j == i)
                cluster_points = np.array(x[cluster_idx])
                self.w_j[i] = np.mean(cluster_points, axis = 0)
                new_error += ((cluster_points - self.w_j[i])**2).sum()
        
            if (previous_error <= new_error):
                print("E NO LONGER DECREASES")
                break

            previous_error = new_error
            iterations += 1

        print("TOTAL ITERATIONS: %d" % iterations)

    def predict (self, x, k):
        y_pred = np.empty(x.shape[0])
        for i in range(x.shape[0]):
            y_pred[i] = self.findClosestPrototype(x[i], self.w_j, k)
            
        return y_pred

            

In [10]:
# initialize and train the model
model2 = KMeans()
model2.train(x_train, y_train, x_test, y_test, 3)

E NO LONGER DECREASES
TOTAL ITERATIONS: 3


In [11]:
# evaluate the model and print the confusion matrixes for both training and test sets
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import mode

#usually we use the Kmeans algorithm when labels are unknown
#here, the labels were only used to evaluate the model's
#performance. the classification report/confustion matrix generated here
#don't make sense in a real-world scenario

print("TRAINING SET")
y_pred1 = model2.predict(x_train, 3)
print(confusion_matrix(y_train, y_pred1))
print(classification_report(y_train, y_pred1))

print("TESTING SET")
y_pred2 = model2.predict(x_test, 3)
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))


# we could also match each learned cluster with a truth label using scipy.stats mode function
def matchLabel(y_test, y_pred, k):
     predictions = np.zeros_like(y_pred)
     for i in range(k):
          clus_idx = np.where(y_pred == i) #return all index that have value of a cluster 
          predictions[clus_idx] = mode(y_test[clus_idx], keepdims=True)[0] #ground (truth) value for a cluster is the mode of the label

     print(confusion_matrix(y_test, predictions))
     print(classification_report(y_test, predictions, zero_division = 1))


print("TRAINING SET WITH MATCHED LABELS")
matchLabel(y_train, y_pred1, 3)
print("TESTING SET WITH MATCHED LABELS")
matchLabel(y_test, y_pred2, 3)

TRAINING SET
[[ 60   0  47]
 [ 32   0  26]
 [  0 101   0]]
              precision    recall  f1-score   support

           0       0.65      0.56      0.60       107
           1       0.00      0.00      0.00        58
           2       0.00      0.00      0.00       101

    accuracy                           0.23       266
   macro avg       0.22      0.19      0.20       266
weighted avg       0.26      0.23      0.24       266

TESTING SET
[[13  0 26]
 [ 2  0  8]
 [ 0 18  0]]
              precision    recall  f1-score   support

           0       0.87      0.33      0.48        39
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        18

    accuracy                           0.19        67
   macro avg       0.29      0.11      0.16        67
weighted avg       0.50      0.19      0.28        67

TRAINING SET WITH MATCHED LABELS
[[107   0   0]
 [ 58   0   0]
 [  0   0 101]]
              precision    recall  f1-score   suppo